In [5]:
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.agents import initialize_agent, AgentType
from dotenv import load_dotenv
import os

# 加载环境变量（请将你的 OpenAI API key 存入 .env 文件）
load_dotenv()

# 定义工具函数
@tool("delete_file", return_direct=True)
def delete_file(filename: str) -> str:
    """删除指定文件"""
    import os
    if not os.path.exists(filename):
        return f"文件 {filename} 不存在。"
    os.remove(filename)
    return f"已删除文件 {filename}。"

@tool("create_file", return_direct=True)
def create_file(filename: str, content: str = "") -> str:
    """创建新文件并写入内容"""
    import os
    if os.path.exists(filename):
        return f"文件 {filename} 已存在。"
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(content)
    return f"已创建文件 {filename}。"

# 初始化 LLM
chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.7,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

# 注册工具
tools = [delete_file, create_file]

# 初始化 agent（function calling agent）
agent = initialize_agent(
    tools,
    chat,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

# 示例对话
result = agent.run("请创建文件 demo.txt 内容为 hello world")
print(result)

result = agent.run("删除文件 demo.txt")
print(result) 



> Entering new AgentExecutor chain...

Invoking: `create_file` with `{'filename': 'demo.txt', 'content': 'hello world'}`


已创建文件 demo.txt。


> Finished chain.
已创建文件 demo.txt。


> Entering new AgentExecutor chain...

Invoking: `delete_file` with `{'filename': 'demo.txt'}`


已删除文件 demo.txt。


> Finished chain.
已删除文件 demo.txt。
